<a href="https://colab.research.google.com/github/Belvinbarasa/Eng.barasa/blob/main/BELVIN_BARASA_WANMI_CHATBOT_ASSIGNMENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install NLTK
!pip install nltk

In [ ]:
# 2. Imports
import json
import random
import numpy as np
import nltk
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# 3. Define intents for a mechanical engineering firm
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hello", "Hi", "Good morning", "Good afternoon", "Hey"],
            "responses": ["Hello! How can I assist you today?", "Hi there! What can I do for you?"]
        },
        {
            "tag": "services",
            "patterns": ["What services do you offer?", "Tell me about your services", "Do you provide CAD design?", "Do you offer mechanical design?"],
            "responses": ["We offer CAD design, FEA analysis, prototyping, and manufacturing solutions."]
        },
        {
            "tag": "maintenance",
            "patterns": ["Do you offer equipment maintenance?", "Can you repair machines?", "What about maintenance services?"],
            "responses": ["Yes, we offer scheduled and emergency maintenance for industrial machinery."]
        },
        {
            "tag": "design",
            "patterns": ["Do you do 3D modeling?", "Can you design mechanical parts?", "I need a part designed"],
            "responses": ["Absolutely! We specialize in 3D modeling and mechanical component design."]
        },
        {
            "tag": "contact",
            "patterns": ["How can I contact you?", "What's your phone number?", "Email address please", "Where are you located?"],
            "responses": ["You can contact us at info@mechengfirm.com or call us at (123) 456-7890."]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you later", "Thanks, bye"],
            "responses": ["Goodbye! Feel free to reach out if you have more questions.", "Bye! Have a great day!"]
        }
    ]
}

In [ ]:
# 4. Prepare the data
all_patterns = []
all_tags = []

for intent in intents['intents']:
    for pattern in intent['patterns']:
        all_patterns.append(pattern)
        all_tags.append(intent['tag'])

# Tokenize
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(all_patterns)
sequences = tokenizer.texts_to_sequences(all_patterns)
padded_sequences = pad_sequences(sequences, padding='post')

# Label encode
lbl_encoder = LabelEncoder()
labels = lbl_encoder.fit_transform(all_tags)

In [ ]:
# 5. Build the GRU model
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16
max_length = padded_sequences.shape[1]
num_classes = len(set(labels))

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    GRU(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 6. Train the model
model.fit(padded_sequences, np.array(labels), epochs=300, verbose=0)

In [ ]:
# 7. Chat function
def chat():
    print("Welcome to the Mechanical Engineering Assistant! Type 'quit' to exit.")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            print("Bot: Goodbye! Have a great day.")
            break
        seq = tokenizer.texts_to_sequences([inp])
        padded = pad_sequences(seq, maxlen=max_length, padding='post')
        pred = model.predict(padded, verbose=0)
        tag = lbl_encoder.inverse_transform([np.argmax(pred)])[0]
        for intent in intents['intents']:
            if intent['tag'] == tag:
                print("Bot:", random.choice(intent['responses']))
                break

In [ ]:

# 8. Run it
chat()

Welcome to the Mechanical Engineering Assistant! Type 'quit' to exit.
Bot: Hi there! What can I do for you?
Bot: We offer CAD design, FEA analysis, prototyping, and manufacturing solutions.
Bot: Yes, we offer scheduled and emergency maintenance for industrial machinery.
